In [0]:
# ======================================
# SILVER TRANSFORMATION - CUSTOMERS
# ======================================
# Objetivo:
# Tomar datos crudos desde la capa Bronze (bronze_customers)
# y generar una tabla Silver con:
# - Tipos de datos correctos
# - Texto normalizado
# - Duplicados eliminados
# - Datos listos para consumo analítico / dbt
# ======================================


# -----------------------------
# 1. Imports necesarios
# -----------------------------
from pyspark.sql.functions import (
    col,
    lower,
    trim,
    to_date,
    row_number
)

from pyspark.sql.window import Window

In [0]:
# -----------------------------
# 2. Definición de catálogo y esquema
# -----------------------------
# Usamos Unity Catalog (workspace) y el esquema del proyecto
# para mantener orden y consistencia

CATALOG = "workspace"
SCHEMA = "retail_medallion_pipeline_schema"

BRONZE_TABLE = f"{CATALOG}.{SCHEMA}.bronze_customers"
SILVER_TABLE = f"{CATALOG}.{SCHEMA}.silver_customers"



In [0]:
# -----------------------------
# 3. Lectura de datos Bronze
# -----------------------------
# Bronze representa la data cruda sin transformar

df_bronze = spark.table(BRONZE_TABLE)

# Visualización opcional para inspección
display(df_bronze)



In [0]:
# -----------------------------
# 4. Transformaciones Silver
# -----------------------------
# En esta capa:
# - Limpiamos strings
# - Normalizamos emails
# - Convertimos fechas
# - Eliminamos duplicados


# 1️⃣ Transformaciones de limpieza y tipado
df_clean = (
    df_bronze
    # Limpieza de campos de texto (elimina espacios en blanco)
    .withColumn("customer_id", trim(col("customer_id")))
    .withColumn("first_name", trim(col("first_name")))
    .withColumn("last_name", trim(col("last_name")))
    .withColumn("email", lower(trim(col("email"))))  # normaliza a minúsculas

    # Conversión de tipo de fecha (string → date)
    .withColumn("signup_date", to_date(col("signup_date")))
)

# 2️⃣ Ventana para deduplicación con criterio de negocio
# Nos quedamos con el registro más reciente por customer_id
window_spec = (
    Window
    .partitionBy("customer_id")
    .orderBy(col("signup_date").desc())
)

# 3️⃣ Eliminación de duplicados usando criterio explícito
df_silver = (
    df_clean
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)


In [0]:
# -----------------------------
# 5. Validaciones básicas
# -----------------------------
# Conteo total de registros
print("Total registros Silver:", df_silver.count())

# Verificación de claves nulas
print(
    "Registros con customer_id nulo:",
    df_silver.filter(col("customer_id").isNull()).count()
)




In [0]:
# -----------------------------
# 6. Escritura de tabla Silver
# -----------------------------
# Guardamos la tabla en formato Delta dentro del esquema del proyecto

(
    df_silver.write
    .format("delta")
    .mode("overwrite") #para sobre escribir la tabla existente
    .saveAsTable(SILVER_TABLE)
)




In [0]:
# -----------------------------
# 7. Verificación final
# -----------------------------
# Confirmamos que la tabla Silver se creó correctamente

spark.table(SILVER_TABLE).display()